In [174]:
def collect_data(ticker):
  ## necessary libraries
  import requests, pandas as pd, datetime, time

  columns = ['v','vw','o','c','h','l','n']
  df = pd.DataFrame(columns=columns)

  api_key = '99886PpFI66KHkpCz9rpp_tL4VoQeSfz'
  base_url = 'https://api.polygon.io/v2/'

  start = datetime.datetime.today() - datetime.timedelta(days=729)
  temp = start + datetime.timedelta(days=100)

  while start < datetime.datetime.today():
    # api endpoint
    endpoint = f"aggs/ticker/{ticker}/range/1/minute/{start.strftime('%Y-%m-%d')}/{temp.strftime('%Y-%m-%d')}?adjusted=true&sort=asc&limit=50000&apiKey=99886PpFI66KHkpCz9rpp_tL4VoQeSfz"

    # make a GET request
    response = requests.get(f'{base_url}{endpoint}', headers={'Authorization': f'Bearer {api_key}'})

    # attempt to add response data to dataframe
    try:
      df_temp = pd.DataFrame(response.json()["results"])
      df_temp['t'] = pd.to_datetime(df_temp['t'], unit='ms')
      df_temp.set_index('t', inplace=True)
      df = pd.concat([df,df_temp],ignore_index=False)
    except: pass
    start = temp
    temp = start + datetime.timedelta(days=100)
    time.sleep(3) # for timing of API calls
    # print df_temp (debug statement)
  
  # clean dataframe
  df = df[~df.index.duplicated(keep='first')]
  print("Generated",len(df),"datapoints")
  return df

def add_to_database(ticker,df,overwrite):
  ## necessary libraries
  import sqlite3, os

  # create folder if it doesn't exist
  price_data_folder = f'stock_data/{ticker}'
  if not os.path.exists(price_data_folder):
    os.makedirs(price_data_folder)
    
  # establish connection to sql database
  connection = sqlite3.connect(f'{price_data_folder}/price_data.db')
  cursor_price_data = connection.cursor()

  # create database if it does not exist
  create_price_data_query = """
  CREATE TABLE IF NOT EXISTS STOCKDATA (
    Timestamp INTEGER PRIMARY KEY,
    OpenPrice REAL,
    HighPrice REAL,
    LowPrice REAL,
    ClosePrice REAL,
    Volume INTEGER,
    Transactions INTEGER,
    VWAP REAL
  );
  """
  # remove data db if overwrite
  if overwrite:
    os.remove(f"{price_data_folder}/price_data.db")
    connection = sqlite3.connect(f'{price_data_folder}/price_data.db')
    cursor_price_data = connection.cursor()

  # commit changes to db
  cursor_price_data.execute(create_price_data_query)
  connection.commit()

  ## get stock data and store to db
  for index, row in df.iterrows():
    timestamp = int(index.timestamp())
    open_price = row['o']
    high_price = row['h']
    low_price = row['l']
    close_price = row['c']
    volume = row['v']
    transactions = row['n']
    vwap = row['vw']

    insert_query = """
    INSERT INTO STOCKDATA (Timestamp, OpenPrice, HighPrice, LowPrice, ClosePrice, Volume, Transactions, VWAP)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?);
    """

    cursor_price_data.execute(insert_query, (timestamp, open_price, high_price, low_price, close_price, volume, transactions, vwap))

  cursor_price_data.close()
  connection.close()

  connection = sqlite3.connect(f'stock_data/{ticker}/info.db')

  cursor_info = connection.cursor()

  create_info_query = """
  CREATE TABLE IF NOT EXISTS INFO (
    StockTicker TEXT,
    StartTime INTEGER,
    EndTime INTEGER,
    IsOTC BOOLEAN
  );
  """

  # remove info db if overwrite
  if overwrite:
    os.remove(f"{price_data_folder}/info.db")
    connection = sqlite3.connect(f'{price_data_folder}/info.db')
    cursor_info = connection.cursor()

  cursor_info.execute(create_info_query)

  connection.commit()

  ## get stock data and store to DB

  cursor_info.close()
  connection.close()

def create_database(ticker,overwrite=True):
  df = collect_data(ticker)
  add_to_database(ticker,df,overwrite)
  

In [175]:
create_database("AMZN",overwrite=True)
create_database("AAPL",overwrite=True)
create_database("NVDA",overwrite=True)

Generated 231282 datapoints
Generated 113904 datapoints
Generated 100000 datapoints
